# Число смертей, связанных с неинфекционными заболеваниями, разбитые по типу заболевания

In [1]:
import pandas as pd

from warnings import simplefilter
simplefilter('ignore')

Эти данные можно найти на сайте ВОЗ по ссылке \
https://www.who.int/data/gho/data/indicators/indicator-details/GHO/number-of-deaths-attributed-to-non-communicable-diseases-by-type-of-disease-and-sex

In [2]:
# Загрузим таблицу
data = pd.read_csv('../data/who_adult_ncd_causes_mortality.csv')

data.head(2)

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,SDG_SH_DTH_RNCOM,Number of deaths attributed to non-communicabl...,numeric,EUR,Europe,Country,MNE,Montenegro,Year,2019,...,NaN,NaN,63.92,NaN,145.0,100 [64-145],NaN,NaN,EN,2021-02-08T21:00:00.000Z
1,SDG_SH_DTH_RNCOM,Number of deaths attributed to non-communicabl...,numeric,EUR,Europe,Country,RUS,Russian Federation,Year,2019,...,NaN,NaN,797625.00,NaN,1203905.0,1003695 [797625-1203905],NaN,NaN,EN,2021-02-08T21:00:00.000Z


Описание нужных нам столбцов.

- `SpatialDimValueCode` - код страны
- `Period` - год
- `Dim2` - код причины смерти
- `Dim2ValueCode` - описание причины
- `FactValueNumeric` - количество смертей

Оставим только их.

In [3]:
columns = [
    'SpatialDimValueCode',
    'Period',
    'Dim2',
    'Dim2ValueCode',
    'FactValueNumeric',
]

data = data[columns]

data.head()

,SpatialDimValueCode,Period,Dim2,Dim2ValueCode,FactValueNumeric
0,MNE,2019,Respiratory diseases,GHECAUSES_GHE117,100.0
1,RUS,2019,Cardiovascular diseases,GHECAUSES_GHE110,1003695.0
2,URY,2019,Diabetes mellitus,GHECAUSES_GHE080,1004.0
3,BRB,2019,Cardiovascular diseases,GHECAUSES_GHE110,1005.0
4,MAR,2019,Respiratory diseases,GHECAUSES_GHE117,10054.0


In [4]:
# Проверим, что пропусков нет
data.isna().sum().sum()

0

In [5]:
print(f"Диапазон лет: {data['Period'].min()} - {data['Period'].max()} гг.")

Диапазон лет: 2000 - 2019 гг.


За 2020г. данных нет. Заполним их средним за последние 3 года по стране.

In [6]:
# Составим список с кодами причин смертности
cause_codes = list(data['Dim2ValueCode'].unique())
print('Итоговое количество причин', len(cause_codes))

# Составим список кодов стран
location_codes = list(data['SpatialDimValueCode'].unique())
print('Итоговое количество стран', len(location_codes))

Итоговое количество причин 4
Итоговое количество стран 183


Создадим таблицу, в которой значения смертности сделаем столбцами.

In [7]:
cause_table_columns = ['SpatialDimValueCode', 'Period'] + cause_codes

causes_data = pd.DataFrame(
    columns=cause_table_columns
)
causes_data

,SpatialDimValueCode,Period,GHECAUSES_GHE117,GHECAUSES_GHE110,GHECAUSES_GHE080,GHECAUSES_GHE061


In [8]:
# Заполним таблицу
def get_cause_value(SpatialDimValueCode: str, Period: int, Dim2ValueCode: str) -> int:
    """Возвращает число смертей для переданных параметров

    Args:
        SpatialDimValueCode (str): код страны
        Period (int): год
        Dim2ValueCode (str): код причины

    Returns:
        int: количество смертей для переданных параметров
    """
    # Создаем маску для переданных параметров
    mask = (data['SpatialDimValueCode'] == SpatialDimValueCode) & \
        (data['Period'] == Period) & \
        (data['Dim2ValueCode'] == Dim2ValueCode)
        
    # Вернем значение для переданных параметров
    return round(data[mask]['FactValueNumeric'].values[0])


# Цикл по годам
for year in range(2000, 2020):
    # Создадим пустую таблицу для данного года
    year_data = pd.DataFrame(columns=cause_table_columns)
    # Заполним столбец с кодами стран
    year_data['SpatialDimValueCode'] = location_codes
    # Заполним столбец с годом
    year_data['Period'] = year
    
    # Теперь заполним столбцы с причинами    
    for cause_field in cause_codes:
        year_data[cause_field] = year_data.apply(
            lambda x: get_cause_value(
                x['SpatialDimValueCode'],
                year,
                cause_field,
            ), axis=1
        )
    
    # Присоединим полученную таблицу к основной
    causes_data = pd.concat(
        [causes_data, year_data], 
        ignore_index=True
    )

# Посмотрим на результат
causes_data.head()

,SpatialDimValueCode,Period,GHECAUSES_GHE117,GHECAUSES_GHE110,GHECAUSES_GHE080,GHECAUSES_GHE061
0,MNE,2000,68,3269,153,960
1,RUS,2000,59915,1265019,12976,323636
2,URY,2000,2104,10592,694,7947
3,BRB,2000,108,847,232,556
4,MAR,2000,6019,71045,4161,16699


Создадим таблицу с описанием кодов причин смертности.

In [9]:
cause_codes_data = data.groupby(['Dim2ValueCode']).first().reset_index()
cause_codes_data = cause_codes_data[['Dim2ValueCode', 'Dim2']]
cause_codes_data

,Dim2ValueCode,Dim2
0,GHECAUSES_GHE061,Malignant neoplasms
1,GHECAUSES_GHE080,Diabetes mellitus
2,GHECAUSES_GHE110,Cardiovascular diseases
3,GHECAUSES_GHE117,Respiratory diseases


Переименуем столбцы.

In [10]:
cause_codes_data.rename(
    columns={ 'Dim2ValueCode': 'WhoCauseCode', 'Dim2': 'Description' },
    inplace=True,
)
cause_codes_data

,WhoCauseCode,Description
0,GHECAUSES_GHE061,Malignant neoplasms
1,GHECAUSES_GHE080,Diabetes mellitus
2,GHECAUSES_GHE110,Cardiovascular diseases
3,GHECAUSES_GHE117,Respiratory diseases


In [11]:
# Соберем словарь с соответствием кода причины и имен новых столбцов
rename_dict = {
    'GHECAUSES_GHE061': 'AdultNcdMortality061',
    'GHECAUSES_GHE080': 'AdultNcdMortality080',
    'GHECAUSES_GHE110': 'AdultNcdMortality110',
    'GHECAUSES_GHE117': 'AdultNcdMortality117',
}

# Добавим имя нового столбца в таблицу
cause_codes_data['CauseCode'] = cause_codes_data['WhoCauseCode'].apply(
    lambda who_code: rename_dict[who_code]
)
cause_codes_data

,WhoCauseCode,Description,CauseCode
0,GHECAUSES_GHE061,Malignant neoplasms,AdultNcdMortality061
1,GHECAUSES_GHE080,Diabetes mellitus,AdultNcdMortality080
2,GHECAUSES_GHE110,Cardiovascular diseases,AdultNcdMortality110
3,GHECAUSES_GHE117,Respiratory diseases,AdultNcdMortality117


In [12]:
# Сохраним полученную таблицу
cause_codes_data.to_csv('../data/adult_ncd_mortality_codes.csv', index=False)

Теперь переименуем столбцы в таблице с данными о смертности.

In [13]:
# В нашей таблице переименуем столбцы 
causes_data.rename(
    columns=rename_dict,
    inplace=True,
)
causes_data.head(3)

,SpatialDimValueCode,Period,AdultNcdMortality117,AdultNcdMortality110,AdultNcdMortality080,AdultNcdMortality061
0,MNE,2000,68,3269,153,960
1,RUS,2000,59915,1265019,12976,323636
2,URY,2000,2104,10592,694,7947


Заполним данные за 2020 г. средним за последние 3 года по стране.

In [14]:
# Маска для данных за последние 3 года
mask = causes_data['Period'].isin([2019, 2018, 2017])

# Группируем по коду страны и берем среднее по всем столбцам
causes_mean_last_3_year_data = causes_data[mask]\
    .groupby('SpatialDimValueCode').agg('mean')

# Сбросим индекс
causes_mean_last_3_year_data.reset_index(inplace=True)

# Заполним столбец с годом
causes_mean_last_3_year_data['Period'] = 2020

causes_mean_last_3_year_data.head()

,SpatialDimValueCode,Period,AdultNcdMortality117,AdultNcdMortality110,AdultNcdMortality080,AdultNcdMortality061
0,AFG,2020,8084.0,68961.333333,7707.0,15023.0
1,AGO,2020,4287.0,26775.666667,4917.333333,10061.333333
2,ALB,2020,1066.0,17848.333333,247.0,4591.0
3,ARE,2020,1413.666667,7193.0,1597.666667,1981.333333
4,ARG,2020,36704.666667,97137.0,9717.0,66041.0


Присоединим полученные данные за 2020 г. к основной таблице.

In [15]:
causes_data = pd.concat([
    causes_data,
    causes_mean_last_3_year_data,
], ignore_index=True)

causes_data.head()

,SpatialDimValueCode,Period,AdultNcdMortality117,AdultNcdMortality110,AdultNcdMortality080,AdultNcdMortality061
0,MNE,2000,68,3269,153,960
1,RUS,2000,59915,1265019,12976,323636
2,URY,2000,2104,10592,694,7947
3,BRB,2000,108,847,232,556
4,MAR,2000,6019,71045,4161,16699


Теперь посчитаем сумму по всем причинам.

In [16]:
causes_data['AdultNcdMortalitySum'] = \
    causes_data[list(rename_dict.values())].sum(axis=1)

causes_data.head()

,SpatialDimValueCode,Period,AdultNcdMortality117,AdultNcdMortality110,AdultNcdMortality080,AdultNcdMortality061,AdultNcdMortalitySum
0,MNE,2000,68,3269,153,960,4450
1,RUS,2000,59915,1265019,12976,323636,1661546
2,URY,2000,2104,10592,694,7947,21337
3,BRB,2000,108,847,232,556,1743
4,MAR,2000,6019,71045,4161,16699,97924


In [17]:
# Сохраним полученную таблицу
causes_data.to_csv('../data/who_adult_ncd_causes_mortality_prepare.csv', index=False)